In [1]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import legacy
from sklearn.metrics import classification_report, confusion_matrix
import cv2

In [3]:
dataset_dir = 'private/tomatect'

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [5]:
num_classes = 5

In [6]:
k = 5

In [7]:
image_paths = []
labels = []

In [8]:
for class_folder in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_folder)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image_paths.append(image_path)
            labels.append(class_folder)

In [9]:
image_paths = np.array(image_paths)
labels = np.array(labels)

In [10]:
classes, class_indices = np.unique(labels, return_inverse=True)

In [11]:
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [12]:
accuracies = []
recalls = []
precisions = []


In [13]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
    ]
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-14 21:18:54.322762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-14 21:18:54.323168: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:

model = tf.keras.Sequential([
      data_augmentation,
      tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

In [15]:
image_width = 150
image_height = 150

In [19]:
for train_index, test_index in kf.split(image_paths, class_indices):
    X_train_paths, X_test_paths = image_paths[train_index], image_paths[test_index]
    y_train, y_test = class_indices[train_index], class_indices[test_index]

    # Load and preprocess images for training
    X_train = []
    for image_path in X_train_paths:
      image = cv2.imread(image_path)
      if image is None:
        print("Failed to load image:", image_path)
      else:
        # Resize and preprocess the image
        preprocessed_image = cv2.resize(image, (image_width, image_height))
        preprocessed_image = image / 255.0  # Normalize the image
        X_train.append(preprocessed_image)
    X_train = np.array(X_train)

    # Load and preprocess images for testing
    X_test = []
    for image_path in X_test_paths:
      image = cv2.imread(image_path)
      if image is None:
        print("Failed to load image:", image_path)
      else:
        # Resize and preprocess the image
        preprocessed_image = cv2.resize(image, (image_width, image_height))
        preprocessed_image = image / 255.0  # Normalize the image
        X_test.append(preprocessed_image)
    X_test = np.array(X_test)
    
    # Convert class labels to one-hot encoded vectors
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

X_train.shape()
y_train.shape()
  

Failed to load image: private/tomatect/spider_mites/.DS_Store
Failed to load image: private/tomatect/leaf_mold/.DS_Store
